In [ ]:
import pandas as pd
import psycopg2
import os

# Outside of K8S cluster

In [ ]:
"""
First, make sure that the database cluster is running inside the k8s cluster

Choose the service using Principle of Least Privilege (e.g. use read only if you are not writing to the db):

# Access the read write instance (Writer instance)
>>> kubectl port-forward svc/sensor-db-ha-rw 5432:5432 

# Access the read only instance (Reader instances)
>>> kubectl port-forward svc/sensor-db-ha-ro 5432:5432 

You need to run the port-forward command to be able to access the database outside of the cluster (e.g. in your local environment)

# Use dataset_download.sh to reset the database data (go back to data in the csvs only)
>>> ./scripts/dataset_download.sh 

# Make sure you have already set up the postgres password in your environment variables
>>> env | grep POSTGRES_PASS
"""

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.
E0130 12:58:46.098944   36664 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp [::1]:8080: connectex: No connection could be made because the target machine actively refused it."
E0130 12:58:46.124431   36664 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp [::1]:8080: connectex: No connection could be made because the target machine actively refused it."
E0130 12:58:46.125477   36664 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp [::1]:8080: connectex: No connection could be made because the target machine actively refused it."
E0130 12:58:46.126533   36664 memcache.go:265] "Unhandled Error" err="couldn't get current serv

In [ ]:
# This will automatically take the db password from the env variables
# Note that you need to run the python scripts inside the system where you set the env var (e.g WSL)
POSTGRES_PASS = os.getenv('POSTGRES_PASS')
conn = psycopg2.connect(
    user="admin",
    password=POSTGRES_PASS,
    host="127.0.0.1",
    port="5432",
    database="sensor-db"
)
cursor = conn.cursor()

# Check to see if connection was successful
cursor.execute("SELECT version();")
record = cursor.fetchone()
print(f"You are connected to: {record}")

You are connected to: ('PostgreSQL 18.0 (Debian 18.0-1.pgdg11+3) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


In [8]:
# Show all tables in the database
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE' AND table_schema NOT IN ('pg_catalog', 'information_schema', 'pg_toast');")
print(cursor.fetchall())

[('iot_pond_1',), ('iot_pond_12',), ('iot_pond_4',), ('iot_pond_8',), ('iot_pond_10',), ('iot_pond_2',), ('iot_pond_6',), ('iot_pond_9',), ('iot_pond_11',), ('iot_pond_3',), ('iot_pond_7',)]


Now you can send sql queries to the database using cursor.execute(sql)

In [ ]:
# You can also use pandas, but you will need sqlalchemy
from sqlalchemy import create_engine

POSTGRES_PASS = os.getenv('POSTGRES_PASS')
DB_NAME = "sensor-db"
USER = "admin"
PASSWORD = POSTGRES_PASS
HOST = "127.0.0.1"
PORT = "5432"

engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}')

df = pd.read_sql("SELECT * FROM iot_pond_1", engine)
df.head()

,created_at,entry_id,temperature,turbidity,dissolved_oxygen,ph,ammonia,nitrate,population,fish_length,fish_weight
0,2021-06-18 23:00:05+00:00,1889,24.8750,100.0,4.505,8.43365,0.45842,193.0,50,7.11,2.91
1,2021-06-18 23:01:02+00:00,1890,24.9375,100.0,6.601,8.43818,0.45842,194.0,50,7.11,2.91
2,2021-06-18 23:01:22+00:00,1891,24.8750,100.0,15.797,8.42457,0.45842,192.0,50,7.11,2.91
3,2021-06-18 23:01:44+00:00,1892,24.9375,100.0,5.046,8.43365,0.45842,193.0,50,7.11,2.91
4,2021-06-18 23:02:07+00:00,1893,24.9375,100.0,38.407,8.40641,0.45842,192.0,50,7.11,2.91


# Inside of K8S cluster

For accessing the database from a pod inside the cluster, use the ClusterIP DNS service as the hostname instead of 127.0.0.1. Everything else is the same

`sensor-db-ha-rw` (Read/Write tasks, writer instance only)

`sensor-db-ha-ro` (Read/Only tasks, reader instances only)

`sensor-db-ha-r`  (Read/Only tasks, but reads from reader and writer instances)

Port: 5432